In [1]:
from symulathon import Simulation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm.notebook import tqdm
from simulation_functions import simulate
%matplotlib notebook

In [2]:
sim = Simulation(1, 1)
nDoF = sim.getDoF()
mass = sim.getMassMatrix()
h = sim.getTimeStep()
DIFF_FRAMES = 100

In [3]:
n_points = 21
k_values = np.linspace(0.01, 6, n_points)
k_bend_values = np.linspace(0, 2, n_points-1)
# k_values = np.linspace(3.95, 4.05, n_points)
# k_bend_values = np.linspace(0, 0.2, n_points)
X, Y = np.meshgrid(k_values, k_bend_values)

g_values = X.tolist()
dgdk_values = X.tolist()
dgdk_bend_values = X.tolist()
ones = np.zeros(X.shape)

# Computation of the surface and gradients
for i in tqdm(range(len(k_values))):
    for j in range(len(k_bend_values)):
        bp = simulate(k_values[i], k_bend_values[j], DIFF_FRAMES)
        dgdp = bp.get_dgdp()
        dgdk_values[j][i] = dgdp[0]
        dgdk_bend_values[j][i] = dgdp[1]
        g_values[j][i] = bp.get_g()

# Finite differences
dk = k_values[1]-k_values[0]
dk_bend = k_bend_values[1]-k_bend_values[0]
dgdk_bend_values_finite, dgdk_values_finite = np.gradient(g_values, dk_bend, dk)

  0%|          | 0/21 [00:00<?, ?it/s]

In [6]:
# Plotting
# 3D plot
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
g_values = np.array(g_values)
dgdk_values = np.array(dgdk_values)
dgdk_bend_values = np.array(dgdk_bend_values)
diff = np.sqrt(
    (dgdk_values - dgdk_values_finite)**2 +
    (dgdk_bend_values - dgdk_bend_values_finite)**2)
dfdp_magnitude_finite = np.sqrt(
    dgdk_values_finite**2 + dgdk_bend_values_finite**2)
diff_rel = diff / np.max(dfdp_magnitude_finite)

colors = cm.Greys(diff)
# surf = ax.plot_surface(X, Y, g_values, facecolors=colors)
surf = ax.plot_surface(X, Y, g_values)

gradient_finite = ax.quiver(X, Y, g_values,
                            dgdk_values_finite,
                            dgdk_bend_values_finite,
                            ones,
                            arrow_length_ratio=0.001,
                            color="red",
                            length=0.5,
                            normalize=True)
ax.set(
    xlabel="k values",
    ylabel="k bend values",
    zlabel="loss",
)
# Add a color bar which maps values to colors.
# fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()
plt.contourf(X, Y, g_values, levels=40)
plt.colorbar()
plt.show()

In [7]:
x0 = np.array([1, 1])
MAX_ITERATIONS = 1000
ALPHA = 1e-6
previous_g = 0

In [8]:
for i in range(MAX_ITERATIONS):
    bp = simulate(x0[0], x0[1], DIFF_FRAMES)
    g = bp.get_g()
    diff = g - previous_g
    previous_g = g
    diff_str = ""
    if (diff < 0):
        diff_str = Fore.GREEN + str(diff) + Style.RESET_ALL
    else:
        diff_str = Fore.RED + str(diff) + Style.RESET_ALL
    print(f"Iteration #{i}\t loss = {g:10.4e},",
            "Difference = "+diff_str)
    dgdp = bp.get_dgdp()
    x0 = x0 - ALPHA * dgdp
    print(f"New parameters {x0}")